In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
import pandas as pd
import json

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo Spark Session với cấu hình tương tác với Kafka</span> 

In [2]:
# spark.sparkContext.stop()

#Spark Session creation configured to interact with Kafka
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11d88f23-e7fb-4ec4-89f8-d4049447632b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in

In [3]:
print(spark.version)

3.0.0


In [4]:
# spark = SparkSession.builder.getOrCreate()
# spark.sparkContext.stop()

<span style="color: blue; font-size:20px; font-weight:bold ">Đọc dữ liệu từ Kafka</span> 

In [5]:
#Read data from Kafka
data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "intent-kafka-1:9092")\
  .option("subscribe", "tweets")\
  .option("startingOffsets", "earliest")\
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
  .select("value")

<span style="color: blue; font-size:20px; font-weight:bold ">Ghi dữ liệu streaming vào file text:</span> 

In [6]:
#write streaming data as a text file
data.\
writeStream.\
format("text").\
option("checkpointLocation", "checkpoint/schema").\
option("format", "text").\
option("path", "schema/in").\
outputMode("append").\
start()

<span style="color: blue; font-size:20px; font-weight:bold ">Trích xuất schema bằng cách đọc file đã ghi ở trên</span> 

In [8]:
#extract schema by reading the file written above
smallBatchSchema = spark.read.json("schema/in/*.txt").schema

24/11/21 02:25:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


<span style="color: blue; font-size:20px; font-weight:bold ">Ghi schema vào file JSON</span> 

In [9]:
import os
import json

# Đường dẫn tới thư mục và file
dir_path = "schema/out"
file_path = os.path.join(dir_path, "tweet_schema.json")

# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Ghi schema vào file JSON
with open(file_path, "w") as f:
    json.dump(smallBatchSchema.jsonValue(), f)

print(f"Schema đã được ghi vào {file_path}")


Schema đã được ghi vào schema/out/tweet_schema.json


<span style="color: red; font-size:30px; font-weight:bold ">Mục đích cuối cùng của file</span> 

### Mục đích của file schemagenerator này là đọc dữ liệu streaming từ Kafka, ghi dữ liệu vào file text, trích xuất schema của dữ liệu đó và ghi schema vào file JSON. Việc này giúp lưu trữ schema của dữ liệu để có thể sử dụng lại hoặc phân tích sau này.